In [1]:
# packages
import numpy as np
from amuse.lab import Particles
import matplotlib.pyplot as plt
from amuse.units.constants import G
from amuse.units import units, constants
from amuse.lab import Huayno, nbody_system
from amuse.community.ph4.interface import ph4
from amuse.units import generic_unit_converter
from amuse.ext.orbital_elements import get_orbital_elements_from_binary
from amuse.ext.orbital_elements import new_binary_from_orbital_elements

In [2]:
# Creating a SIMPLE system of the Sun, Jupiter and Ganymede (largest moon) --> begin with Sun and Jupiter
def get_system():
    system = Particles(2)

    # Declaring Sun
    sun = system[0]
    sun.name = 'Sun'
    sun.mass = 1 | units.MSun
    sun.position = (0,0,0) | units.au
    sun.velocity = (0,0,0) | units.kms

    # Declaring Jupiter
    jupiter = system[1]
    jupiter.name = 'Jupiter'
    jupiter.mass = 1 | units.MJupiter
    jupiter.position = (5.2,0,0) | units.au
    jupiter.eccentricity = 0.0
    jupiter.semimajor_axis = 5.2|units.au

    #RH = half-mass radius = radius of sphere enclosing half of the total system mass
    RH_jup = jupiter.semimajor_axis*(1.0-jupiter.eccentricity)*(jupiter.mass/(3*sun.mass))**(1./3.) 

    # Relative velocities
    def relative_orbital_velocity(mass, distance):
        return (G*mass/distance).sqrt()

    # Orbital periods
    def orbital_period(Mtot, a):
        return (((4 * np.pi**2) * a**3)/(constants.G * Mtot)).sqrt()

    # Jupiter velocity
    vorb_j = relative_orbital_velocity(sun.mass+jupiter.mass, jupiter.position.sum())
    jupiter.velocity = (0, 1, 0) * vorb_j

    # Centering the system
    system.move_to_center()

    # Declaring Ganymede
    ganymede = Particles(1)
    ganymede.name = 'Ganymede'
    ganymede.mass = 1.48E23 | units.kg

    #initial parameters --> will depend on those needed for Lidov-Kozai Mechanism to take effect
    ganymede.position = (1070400,0,0) | units.km 
    ganymede.inclination = 60 | units.deg
    ganymede.eccentricity = 0.6
    ganymede.semimajor_axis = 0.1*RH_jup

    # Ganymede velocity
    vorb_g = relative_orbital_velocity(ganymede.mass+jupiter.mass, ganymede.position.sum())
    ganymede.velocity = (0, 1, 0) * vorb_g 

    # Ganymede orbital period
    Porb_g = orbital_period(jupiter.mass+ganymede.mass, ganymede.semimajor_axis)

    # We want Ganymede to orbit around Jupiter --> must add the positions and velocity of Jupiter to the Ganymede's
    ganymede.position += jupiter.position
    ganymede.velocity += jupiter.velocity

    # Adding Ganymede and recentering the system
    system.add_particle(ganymede)
    system.move_to_center()
    
    return system, sun, jupiter, ganymede

In [3]:
# Integrate system over time
def integrate_system(dt, end_time):
    
    system = sun_jup_gan()
    
    convert_nbody = nbody_system.nbody_to_si(system.mass.sum(), system[1].position.length())
    
    gravity = Huayno(convert_nbody)
    gravity.particles.add_particles(system)
    
    ecc = [] #eccentricity
    inc = [] | units.deg #inclination
    time = [] | units.yr
    
    channel = gravity.particles.new_channel_to(system)
    
    while gravity.model_time < end_time:
        
        orbit_ganymede = get_orbital_elements_from_binary(system[1:3], G=constants.G)
        
        gravity.evolve_model(gravity.model_time + (dt))
        channel.copy()
        
        ecc.append(orbit_ganymede[3])
        inc.append(orbit_ganymede[5])
        time.append(gravity.model_time + (dt))
        
    gravity.stop()
    
    return ecc, inc, time

In [ ]:
ecc, inc, time = integrate_system(10|units.day, 100|units.yr)

In [ ]:
# eccentricity as function of the orbital inclination
def plot_track(ecc, inc, time):
    
    #plt.rcdefaults()
    plt.rcParams.update({'font.size': 20})
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[30, 10])
    
    ax1.scatter(time.value_in(units.yr), np.arccos(inc.value_in(units.deg)), color = 'b')
    ax1.set_ylabel('Inclination [deg]')
    ax1.set_xlabel('Time [years]')
    
    ax2.scatter(time.value_in(units.yr), np.array(ecc), color='g', lw=1)
    ax2.set_ylabel('Eccentricity')
    ax2.set_xlabel('Time [years]')
    ax2.set_xlim([0,40])
   
    plt.show()

In [ ]:
plot_track(ecc, inc, time)